In [1]:
import pickle
from copy import deepcopy
from datetime import datetime as DT
from loguru import logger
import pandas as pd
from theano import shared
from pymc_models import PyMCModel
from pymc_models import bayes_nn_model_ARD_1HL_halfCauchy_hyperpriors as bnn_mdl

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [2]:
with open('../PickleJar/DataSets/AphiTrainTestSplitDataSets.pkl', 'rb') as fb:
    datadict = pickle.load(fb)
X_s_train = datadict['x_train_s']
y_train = datadict['y_train']
X_s_test = datadict['x_test_s']
y_test = datadict['y_test']

In [3]:
bands = [411, 443, 489, 510, 555, 670]
# create band-keyed dictionary to contain models
model_dict=dict.fromkeys(bands)

# create theano shared variable
X_shared = shared(X_s_train.values)
y_shared = shared(y_train['log10_aphy%d' % bands[0]].values)
# Fitting aphi411 model:
# Instantiate PyMC3 model with bnn likelihood
for band in bands:
    logger.info("processing aphi{band}", band=band)
    X_shared.set_value(X_s_train.values)
    y_shared.set_value(y_train['log10_aphy%d' % band].values)
    bnn = PyMCModel(bnn_mdl, X_shared, y_shared )
    bnn.model.name = 'bnn_aphy%d' %band
    bnn.fit(n_samples=2000, cores=1, chains=4, tune=10000,
                nuts_kwargs=dict(target_accept=0.95))
    ppc_train_ = bnn.predict(likelihood_name='likelihood')
    waic_train = bnn.get_waic()
    loo_train = bnn.get_loo()
    model_train = deepcopy(bnn.model)
    trace = deepcopy(bnn.trace_)
    run_dict = dict(model_train=model_train, trace=trace,
                    ppc_train=ppc_train_, loo_train=loo_train, waic_train=waic_train)
    X_shared.set_value(X_s_test.values)
    y_shared.set_value(y_test['log10_aphy%d' % band].values)
    model_test = deepcopy(bnn.model)
    ppc_test_ = bnn.predict(likelihood_name='likelihood')
    waic_test = bnn.get_waic()
    loo_test = bnn.get_loo()
    run_dict.update(dict(model_test=model_test, ppc_test=ppc_test_,
                         waic_test=waic_test, loo_test=loo_test))
    model_dict[band] = run_dict

2019-03-11 15:06:19.946 | INFO     | __main__:<module>:11 - processing aphi411
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (4 chains in 1 job)
NUTS: [sd, bias_o, bias_1, wts_1_o_intrmd, wts_i_1_intrmd, hyp_bias_1_sd, hyp_w_1_out_sd, hyp_w_i_1_sd]
100%|██████████| 12000/12000 [58:03<00:00,  3.43it/s]
There were 26 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 44 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 56 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 81 divergences after tuning. Increase `target_accept` or rep

There were 133 divergences after tuning. Increase `target_accept` or reparameterize.
There were 203 divergences after tuning. Increase `target_accept` or reparameterize.
There were 297 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.
100%|██████████| 8000/8000 [00:50<00:00, 156.92it/s]
2019-03-12 10:35:00.745 | INFO     | __main__:<module>:11 - processing aphi670
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (4 chains in 1 job)
NUTS: [sd, bias_o, bias_1, wts_1_o_intrmd, wts_i_1_intrmd, hyp_bias_1_sd, hyp_w_1_out_sd, hyp_w_i_1_sd]
100%|██████████| 12000/12000 [59:49<00:00,  3.29it/s] 
There were 27 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 132 divergences after tuning. Increase `target_accept` or repa

In [5]:
with open('../PickleJar/Results/bnn_model_dict_%s.pkl' %DT.now(), 'wb') as fb:
        pickle.dump(model_dict, fb, protocol=pickle.HIGHEST_PROTOCOL)